In [1]:
import os
import copy
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlp
%matplotlib inline
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM, BartForConditionalGeneration,BartTokenizer
import shap
import scipy as sp
import nlp
import torch
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)
from shap.utils import assert_import, record_import_error, safe_isinstance, make_masks, OpChain, MaskedModel
from shap import maskers, links
from transformers.file_utils import ModelOutput
from shap.utils import cal_conditional_logits

In [2]:
tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6")
model =  BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-xsum-12-6")

In [3]:
dataset = nlp.load_dataset('xsum',split='train')

WARNING - Using custom data configuration default


In [4]:
def f_kwargs(x):
    inputs = tokenizer([x], max_length=1024, return_tensors='pt',min_length=0)
    input_ids=inputs['input_ids'].cuda()
    out=model.generate(input_ids,return_dict=True)
    decoder_inputs=out[:,:-1]
    return {'decoder_inputs':decoder_inputs}

In [5]:
def f_cp(x_batch,**kwargs):
    output_batch=[]
    decoder_inputs=kwargs['decoder_inputs']
    for i,x in enumerate(x_batch):
        inputs = tokenizer([x], max_length=1024, return_tensors='pt',truncation=True)
        input_ids=inputs['input_ids'].cuda()
        conditional_logits = cal_conditional_logits(input_ids,model,tokenizer,decoder_inputs)
        output_batch.append(conditional_logits)
    return np.array(output_batch)

In [6]:
explainer = shap.Explainer(f_cp,tokenizer,model_kwargs=f_kwargs)
explainer.masker.mask_token=None
explainer.masker.mask_token_str=""

explainers.Partition is still in an alpha state, so use with caution...


In [7]:
shap_values = explainer(dataset['document'][60:61])

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Keyword arguments {'min_length': 0} not recognized.


Partition explainer: 2it [03:25, 102.91s/it]                                                                           


In [10]:
def summarize(x,model,tokenizer,mask=None):
    print(f"Masked input: {x}")
    inputs = tokenizer([x], max_length=1024, return_tensors='pt')
    input_ids=inputs['input_ids'].cuda()
    if mask is not None:
        attention_mask=torch.tensor(mask).unsqueeze(0).type(torch.int64)
    if mask:
        summary_ids = model.generate(input_ids,attention_mask=attention_mask)
    else:
        summary_ids = model.generate(input_ids)
    summary=[tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    print(f"summary: {summary[0]}")
    return summary_ids.detach().cpu().numpy()[0,1:-1]

In [11]:
ids=summarize(dataset['document'][60],model,tokenizer)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Masked input: If you pay for your calls on a contract and you do not report the loss immediately, the phone company can hold you liable for a bill of hundreds or even thousands of pounds.It is estimated that tens of thousands of mobiles are lost or stolen every year, and whilst some people constantly use their phone and would notice any loss very quickly, other people may have one only for emergencies or to use occasionally.Anthea Guthrie only uses her mobile from time to time, so when her phone went missing she did not notice for several days, as she told Radio 4's Money Box programme."I believe it probably went missing during a visit to Bristol, that must have been on the Saturday. I realised it was lost on the Monday and rang T-Mobile," she says.T-Mobile sent her a replacement phone and SIM card and she did not think much more about the old phone until she received a text from T-Mobile estimating her monthly bill.Instead of her normal monthly charge of about £20 she was told her bil

In [15]:
tokenizer.convert_ids_to_tokens(int(ids[18]))

'Ġfailed'

In [16]:
shap.plots.text(shap_values[0][:,18],group_threshold=1)